In [1]:
import pandas as pd
import numpy as np

In [2]:
# make sure you have the folder to save reference files in directory
store_folder = "reference"

In [3]:
Industry_dict = {1: 'Manufacturing & Trading',
                 2: 'Retail & Wholesale',
                 3: 'Financial Services',
                 4: 'Professional Services',
                 5: 'Innovation & Technology'}

In [4]:
City_dict = {1: 'Hong Kong', 
            2: 'Macao', 
            3: 'Guangzhou', 
            4: 'Shenzhen', 
            5: 'Foshan', 
            6: 'Dongguan', 
            7: 'Huizhou', 
            8: 'Zhongshan', 
            9: 'Jiangmen', 
            10: 'Zhuhai', 
            11: 'Zhaoqing'}

City_group_dict = {1: 'Hong Kong', 
                2: 'Macao and other GBA cities', #
                3: 'Guangzhou', 
                4: 'Shenzhen', 
                5: 'Foshan', 
                6: 'Dongguan', 
                7: 'Macao and other GBA cities', #
                8: 'Macao and other GBA cities',# 
                9: 'Macao and other GBA cities', #
                10: 'Macao and other GBA cities', #
                11: 'Macao and other GBA cities'#
                }

In [5]:
# import dataset excel xlsx
df = pd.read_excel("test_dataset_2020-05-18.xlsx")
df.head()

,Unnamed: 0,Q01,Q02,Q03A,Q03B,Q04A,Q04B,Q05A,Q05B,Q06A,...,Q22.01,Q22.02,Q22.03,Q22.04,Q22.05,Q22.06,Q22.07,Q22.08,Q22.09,Q23
0,1,11,4,3,2,2,2,2,3,1,...,0,0,0,0,0,0,0,1,0,1
1,2,9,4,2,2,1,2,3,1,1,...,0,1,1,0,0,0,0,0,0,4
2,3,10,3,3,1,1,2,1,2,1,...,0,0,1,0,0,0,0,1,0,3
3,4,4,5,1,2,2,2,2,3,4,...,0,0,0,0,0,1,0,0,0,1
4,5,9,3,1,2,2,2,2,2,3,...,0,0,1,1,0,0,0,0,0,6


#Part 0: Data Preparation - 1)GDP, 2)GBA_Company_DB_size, 3)survey sample size matrices

##Part 0.1: GDP

In [6]:
# import city GDP by industry excel xlsx
GDP_df = pd.read_excel("./metadata/1)citygroupGDP.xlsx",sheet_name='2017CitygroupGDP')
GDP_df

,City_group,Manufacturing & Trading,Retail & Wholesale,Financial Services,Professional Services,Innovation & Technology,Total_GDP
0,Hong Kong,57313,13113,61603,40005,2347.0,341652
1,Guangzhou,80863,46740,28957,24722,25382.0,318480
2,Shenzhen,129434,35064,43311,10383,36529.0,333097
3,Foshan,77469,9495,6040,2921,NaN,139200
4,Dongguan,52849,12860,6778,7021,NaN,112297
5,Macao and other GBA cities,98626,23408,14424,11732,NaN,269009


In [7]:
# get the GDP city column, make sure the order of cities is same as the questionaire
City_group_rows=GDP_df['City_group'].values.tolist()
City_group_rows

['Hong Kong',
 'Guangzhou',
 'Shenzhen',
 'Foshan',
 'Dongguan',
 'Macao and other GBA cities']

##Part 0.2: GBA_TDC_DB_size

In [8]:
# import number_of_establishment_by_city excel xlsx
TDC_DB_size_df = pd.read_excel("./metadata/2)GBA_Company_DB_size.xlsx",sheet_name='TDC_DB_size')
# further make sure establishment matrix's row and column order are in accordance with the GDP one
TDC_DB_size_df=TDC_DB_size_df.loc[TDC_DB_size_df['City_group']==City_group_rows]
TDC_DB_size_df

,City_group,Manufacturing & Trading,Retail & Wholesale,Financial Services,Professional Services,Innovation & Technology
0,Hong Kong,56988,59160,5709,14063,14718.0
1,Guangzhou,38803,20124,844,2349,1662.0
2,Shenzhen,68120,33551,1232,3548,10108.0
3,Foshan,22128,4073,127,398,NaN
4,Dongguan,34668,5532,119,581,NaN
5,Macao and other GBA cities,33953,6630,292,1058,NaN


##Part 0.3: survey_size

In [9]:
# sample size of the GBA survey dataset by city and by industry
survey_size = df[['Q01', 'Q02']]
survey_size=survey_size.rename(columns={'Q01': "City", "Q02": "Industry"})
survey_size.head(10)

,City,Industry
0,11,4
1,9,4
2,10,3
3,4,5
4,9,3
5,9,3
6,1,3
7,6,2
8,4,5
9,11,4


In [10]:
survey_size['City_group'] = survey_size.loc[:,'City']

In [11]:
survey_size.head()

,City,Industry,City_group
0,11,4,11
1,9,4,9
2,10,3,10
3,4,5,4
4,9,3,9


In [12]:
survey_size = survey_size.replace({'City':City_dict,'Industry':Industry_dict,'City_group': City_group_dict})
survey_size.head()

,City,Industry,City_group
0,Zhaoqing,Professional Services,Macao and other GBA cities
1,Jiangmen,Professional Services,Macao and other GBA cities
2,Zhuhai,Financial Services,Macao and other GBA cities
3,Shenzhen,Innovation & Technology,Shenzhen
4,Jiangmen,Financial Services,Macao and other GBA cities


##Part 0.3A: survey size by city

In [13]:
#survey_size = pd.crosstab(survey_size['Q01'],survey_size['Q02']) # <-this line is another function with same result
survey_size_by_city = survey_size.groupby(['City', 'Industry']).size().unstack()

In [14]:
# take a look first
survey_size_by_city

Industry,Financial Services,Innovation & Technology,Manufacturing & Trading,Professional Services,Retail & Wholesale
City,,,,,
Dongguan,13,16,16,22,23
Foshan,23,25,16,16,13
Guangzhou,12,16,10,26,24
Hong Kong,23,14,19,25,13
Huizhou,20,12,22,22,23
Jiangmen,20,12,26,14,16
Macao,22,23,19,23,24
Shenzhen,16,22,13,16,17
Zhaoqing,21,14,15,22,14


In [15]:
survey_size_by_city = survey_size_by_city.loc[:,TDC_DB_size_df.columns[1:]]

In [16]:
# export this survey_size_df to excel format for reference
survey_size_by_city.to_excel(f"{store_folder}/reference_Part0.3A_survey_size_by_city.xlsx", na_rep='N.A.', index=True)

##Part 0.3B: survey size by city group

In [17]:
survey_size_by_city_group = survey_size.groupby(['City_group', 'Industry']).size().unstack()
survey_size_by_city_group

Industry,Financial Services,Innovation & Technology,Manufacturing & Trading,Professional Services,Retail & Wholesale
City_group,,,,,
Dongguan,13,16,16,22,23
Foshan,23,25,16,16,13
Guangzhou,12,16,10,26,24
Hong Kong,23,14,19,25,13
Macao and other GBA cities,125,92,118,112,104
Shenzhen,16,22,13,16,17


In [18]:
# further make sure sample size matrix's row and column order are in accordance with GDP one
survey_size_by_city_group=survey_size_by_city_group.loc[City_group_rows][TDC_DB_size_df.columns[1:]]

In [19]:
survey_size_by_city_group

Industry,Manufacturing & Trading,Retail & Wholesale,Financial Services,Professional Services,Innovation & Technology
City_group,,,,,
Hong Kong,19,13,23,25,14
Guangzhou,10,24,12,26,16
Shenzhen,13,17,16,16,22
Foshan,16,13,23,16,25
Dongguan,16,23,13,22,16
Macao and other GBA cities,118,104,125,112,92


In [20]:
# wipe out industry label
survey_size_by_city_group.columns.name=''

In [21]:
survey_size_by_city_group_df = survey_size_by_city_group.reset_index()
# export this survey_size_df to excel format for reference
survey_size_by_city_group.to_excel(f"{store_folder}/reference_Part0.3B_survey_size_by_city_group.xlsx", na_rep='N.A.', index=True)
survey_size_by_city_group_df

,City_group,Manufacturing & Trading,Retail & Wholesale,Financial Services,Professional Services,Innovation & Technology
0,Hong Kong,19,13,23,25,14
1,Guangzhou,10,24,12,26,16
2,Shenzhen,13,17,16,16,22
3,Foshan,16,13,23,16,25
4,Dongguan,16,23,13,22,16
5,Macao and other GBA cities,118,104,125,112,92


#Part 1: Compile weighted-matrix from survey_size, establishment, GDP

##Part 1.1: Sample-sized weighted-matrix

In [22]:
survey_size_matrix = survey_size_by_city_group_df.values[:,1:]
survey_size_matrix

array([[19, 13, 23, 25, 14],
       [10, 24, 12, 26, 16],
       [13, 17, 16, 16, 22],
       [16, 13, 23, 16, 25],
       [16, 23, 13, 22, 16],
       [118, 104, 125, 112, 92]], dtype=object)

In [23]:
TDC_DB_size_matrix = TDC_DB_size_df.values[:,1:]
TDC_DB_size_matrix

array([[56988, 59160, 5709, 14063, 14718.0],
       [38803, 20124, 844, 2349, 1662.0],
       [68120, 33551, 1232, 3548, 10108.0],
       [22128, 4073, 127, 398, nan],
       [34668, 5532, 119, 581, nan],
       [33953, 6630, 292, 1058, nan]], dtype=object)

In [24]:
# perform element-wise matrix division to obtain SAMPLE_SIZE_weighted_matrix
# to magnify the weighting according to the TDC_DB_size.
SAMPLE_SIZE_weighted_matrix = TDC_DB_size_matrix/survey_size_matrix
SAMPLE_SIZE_weighted_matrix

array([[2999.3684210526317, 4550.7692307692305, 248.2173913043478,
        562.52, 1051.2857142857142],
       [3880.3, 838.5, 70.33333333333333, 90.34615384615384, 103.875],
       [5240.0, 1973.5882352941176, 77.0, 221.75, 459.45454545454544],
       [1383.0, 313.3076923076923, 5.521739130434782, 24.875, nan],
       [2166.75, 240.52173913043478, 9.153846153846153,
        26.40909090909091, nan],
       [287.73728813559325, 63.75, 2.336, 9.446428571428571, nan]],
      dtype=object)

In [25]:
# export this SAMPLE_SIZE_weighted_matrix to excel format for reference
pd.DataFrame(SAMPLE_SIZE_weighted_matrix, columns = TDC_DB_size_df.columns[1:], index = City_group_rows).to_excel(f"{store_folder}/reference_Part1.1_SAMPLE_SIZE_weighted_matrix.xlsx", \
                                                   na_rep='N.A.')

##Part 1.2: GDP-sized weighted-matrix

In [26]:
# have a look again
GDP_df

,City_group,Manufacturing & Trading,Retail & Wholesale,Financial Services,Professional Services,Innovation & Technology,Total_GDP
0,Hong Kong,57313,13113,61603,40005,2347.0,341652
1,Guangzhou,80863,46740,28957,24722,25382.0,318480
2,Shenzhen,129434,35064,43311,10383,36529.0,333097
3,Foshan,77469,9495,6040,2921,NaN,139200
4,Dongguan,52849,12860,6778,7021,NaN,112297
5,Macao and other GBA cities,98626,23408,14424,11732,NaN,269009


In [27]:
# slice industry columns only
GDP_matrix = GDP_df.values[:,1:6]
GDP_matrix

array([[57313, 13113, 61603, 40005, 2347.0],
       [80863, 46740, 28957, 24722, 25382.0],
       [129434, 35064, 43311, 10383, 36529.0],
       [77469, 9495, 6040, 2921, nan],
       [52849, 12860, 6778, 7021, nan],
       [98626, 23408, 14424, 11732, nan]], dtype=object)

In [28]:
Total_City_GDP=sum(GDP_df['Total_GDP'])
Total_City_GDP

1513735

In [29]:
GDP_SIZE_weighted_matrix = GDP_matrix/Total_City_GDP
GDP_SIZE_weighted_matrix

array([[0.03786197716244918, 0.008662678738352485, 0.040696026715376206,
        0.02642800754425312, 0.001550469533967306],
       [0.05341952191103463, 0.03087726715706514, 0.01912950417345176,
        0.016331788589151998, 0.016767796212679233],
       [0.08550637991458214, 0.023163895926301498, 0.028612009367557728,
        0.006859192659217102, 0.02413170072700968],
       [0.051177385737926386, 0.006272564220289548, 0.003990130372885611,
        0.00192966404291372, nan],
       [0.03491298014513769, 0.008495542482667045, 0.004477666170102429,
        0.00463819624967382, nan],
       [0.06515407254241991, 0.015463737047765956, 0.009528748426904312,
        0.007750365817002315, nan]], dtype=object)

In [30]:
# export this GDP_SIZE_weighted_matrix to excel format for reference
pd.DataFrame(GDP_SIZE_weighted_matrix,columns = TDC_DB_size_df.columns[1:], index = City_group_rows).to_excel(f"{store_folder}/reference_Part1.2_GDP_SIZE_weighted_matrix.xlsx", \
                                                   na_rep='N.A.')

##Part 1.3: Calculate the weighted-matrix for each city with industry

In [31]:
# perform "element-wise" matrix multiplication
# beware that here is NOT matrix multiplication, "matrix multiplication" refers to dot-products calculation
# they are 2 different methods, what is using here is "element-wise" matrix multiplication 

WEIGHTED_MATRIX = np.multiply(SAMPLE_SIZE_weighted_matrix, GDP_SIZE_weighted_matrix)
WEIGHTED_MATRIX

array([[113.562018659666, 39.42185185853331, 10.101461587742728,
        14.866282803793265, 1.6299864714950578],
       [207.28377087138767, 25.89058851119912, 1.3454417935327736,
        1.4755142844583862, 1.7417548315920555],
       [448.0534307524104, 45.71599248372597, 2.203124721301945,
        1.5210259721813923, 11.087419588573356],
       [70.7783244755522, 1.9652426207107179, 0.022032459015498808,
        0.04800039306747879, nan],
       [75.64769972947708, 2.0433626527875695, 0.04098786724939915,
        0.12249054641184043, nan],
       [18.74725614434562, 0.9858132367950797, 0.022259156325248473,
        0.073213277092754, nan]], dtype=object)

In [32]:
WEIGHTED_DF = pd.DataFrame(WEIGHTED_MATRIX)
WEIGHTED_DF.columns = TDC_DB_size_df.columns[1:]
WEIGHTED_DF.index=City_group_rows
# illustrate the weighting for each sample by city & industry
WEIGHTED_DF

,Manufacturing & Trading,Retail & Wholesale,Financial Services,Professional Services,Innovation & Technology
Hong Kong,113.562,39.4219,10.1015,14.8663,1.62999
Guangzhou,207.284,25.8906,1.34544,1.47551,1.74175
Shenzhen,448.053,45.716,2.20312,1.52103,11.0874
Foshan,70.7783,1.96524,0.0220325,0.0480004,NaN
Dongguan,75.6477,2.04336,0.0409879,0.122491,NaN
Macao and other GBA cities,18.7473,0.985813,0.0222592,0.0732133,NaN


In [33]:
# export to excel for reference
WEIGHTED_DF.to_excel(f"{store_folder}/reference_Part1.3_weighting_score_by_each_city_industry.xlsx",na_rep='N.A.')

##Part 1.4: Calculate the weighted-matrix for whole dataset

In [34]:
# multiply the WEIGHTED_MATRIX with survey_size_matrix
# This calculate the total weighting for each category in whole survey dataset 
# classified by 2 dimension (City and Industry)
WEIGHTED_SUM_MATRIX = np.multiply(WEIGHTED_MATRIX, survey_size_matrix)
WEIGHTED_SUM_MATRIX

array([[2157.678354533654, 512.484074160933, 232.33361651808275,
        371.6570700948316, 22.819810600930808],
       [2072.837708713877, 621.3741242687788, 16.145301522393282,
        38.36337139591804, 27.868077305472887],
       [5824.6945997813355, 777.1718722233414, 35.24999554083112,
        24.336415554902278, 243.92323094861382],
       [1132.4531916088351, 25.54815406923933, 0.5067465573564726,
        0.7680062890796606, nan],
       [1210.3631956716333, 46.9973410141141, 0.532842274242189,
        2.6947920210604894, nan],
       [2212.1762250327833, 102.52457662668829, 2.782394540656059,
        8.199887034388448, nan]], dtype=object)

In [35]:
WEIGHTED_SUM_score = pd.DataFrame(WEIGHTED_SUM_MATRIX)
WEIGHTED_SUM_score.columns = TDC_DB_size_df.columns[1:]
WEIGHTED_SUM_score.index = City_group_rows

In [36]:
WEIGHTED_SUM_score.to_excel(f"{store_folder}/reference_Part1.4_weighting_SUM_score.xlsx",na_rep='N.A.')
WEIGHTED_SUM_score

,Manufacturing & Trading,Retail & Wholesale,Financial Services,Professional Services,Innovation & Technology
Hong Kong,2157.68,512.484,232.334,371.657,22.8198
Guangzhou,2072.84,621.374,16.1453,38.3634,27.8681
Shenzhen,5824.69,777.172,35.25,24.3364,243.923
Foshan,1132.45,25.5482,0.506747,0.768006,NaN
Dongguan,1210.36,46.9973,0.532842,2.69479,NaN
Macao and other GBA cities,2212.18,102.525,2.78239,8.19989,NaN


In [37]:
#Weighted Total Sum
WEIGHTED_TOTAL_SUM = np.nansum(np.nansum(WEIGHTED_SUM_MATRIX))
WEIGHTED_TOTAL_SUM

17724.484975903968

#Part 2: Compile weighted-index

In [38]:
# just have a look, 5 rows on the dataset
df.head()

,Unnamed: 0,Q01,Q02,Q03A,Q03B,Q04A,Q04B,Q05A,Q05B,Q06A,...,Q22.01,Q22.02,Q22.03,Q22.04,Q22.05,Q22.06,Q22.07,Q22.08,Q22.09,Q23
0,1,11,4,3,2,2,2,2,3,1,...,0,0,0,0,0,0,0,1,0,1
1,2,9,4,2,2,1,2,3,1,1,...,0,1,1,0,0,0,0,0,0,4
2,3,10,3,3,1,1,2,1,2,1,...,0,0,1,0,0,0,0,1,0,3
3,4,4,5,1,2,2,2,2,3,4,...,0,0,0,0,0,1,0,0,0,1
4,5,9,3,1,2,2,2,2,2,3,...,0,0,1,1,0,0,0,0,0,6


##Part 2.1: Convert the data with dictionary {up:1 to 1, same: 2 to 0, down: 3 to -1, N/A: 4 to NA}

In [39]:
conversion_dict ={1 : 1, 2 : 0, 3 : -1, 4: np.nan} 

In [40]:
# Performance sub-index columns
performance_columns = ['Q03A','Q04A','Q05A','Q06A',
                       'Q07A','Q08A','Q09A','Q10A']

In [41]:
# Expectation sub-index columns
expectation_columns = ['Q03B','Q04B','Q05B','Q06B',
                       'Q07B','Q08B','Q09B','Q10B']

In [42]:
# Credit sub-index columns
credit_columns = ['Q13A','Q14A','Q15A','Q16A','Q17A']

In [43]:
# Calculation columns
calculation_columns = []

for i in range(3,18):
    calculation_columns.append(f'Q{i:02}A')
for i in range(3,18):    
    calculation_columns.append(f'Q{i:02}B')
    
print(calculation_columns) 

['Q03A', 'Q04A', 'Q05A', 'Q06A', 'Q07A', 'Q08A', 'Q09A', 'Q10A', 'Q11A', 'Q12A', 'Q13A', 'Q14A', 'Q15A', 'Q16A', 'Q17A', 'Q03B', 'Q04B', 'Q05B', 'Q06B', 'Q07B', 'Q08B', 'Q09B', 'Q10B', 'Q11B', 'Q12B', 'Q13B', 'Q14B', 'Q15B', 'Q16B', 'Q17B']


In [44]:
# Acquire the selected useful columns and then perform conversion
df_converted = df[calculation_columns].replace(conversion_dict)
#df_converted = df.iloc[:,2:].replace(conversion_dict)
df_index = pd.concat([df[['Q01','Q02']], df_converted], sort=False, axis=1)
df_index.head(10)

,Q01,Q02,Q03A,Q04A,Q05A,Q06A,Q07A,Q08A,Q09A,Q10A,...,Q08B,Q09B,Q10B,Q11B,Q12B,Q13B,Q14B,Q15B,Q16B,Q17B
0,11,4,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,NaN,1.0,NaN,-1.0,-1.0,1.0,1.0,-1.0,0.0
1,9,4,0.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,...,1.0,0.0,-1.0,-1.0,NaN,1.0,0.0,-1.0,1.0,1.0
2,10,3,-1.0,1.0,1.0,1.0,-1.0,1.0,1.0,0.0,...,0.0,-1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
3,4,5,1.0,0.0,0.0,NaN,1.0,-1.0,0.0,-1.0,...,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0
4,9,3,1.0,0.0,0.0,-1.0,0.0,1.0,1.0,1.0,...,-1.0,0.0,1.0,-1.0,0.0,-1.0,-1.0,-1.0,1.0,1.0
5,9,3,0.0,-1.0,-1.0,-1.0,0.0,-1.0,NaN,0.0,...,1.0,0.0,0.0,1.0,-1.0,-1.0,-1.0,1.0,0.0,1.0
6,1,3,1.0,0.0,1.0,1.0,-1.0,1.0,NaN,-1.0,...,1.0,0.0,1.0,1.0,NaN,0.0,0.0,1.0,-1.0,1.0
7,6,2,1.0,0.0,-1.0,-1.0,1.0,0.0,-1.0,-1.0,...,1.0,-1.0,-1.0,1.0,NaN,0.0,-1.0,1.0,-1.0,0.0
8,4,5,1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,...,-1.0,0.0,-1.0,1.0,1.0,1.0,-1.0,1.0,0.0,1.0
9,11,4,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,...,0.0,-1.0,1.0,1.0,1.0,-1.0,1.0,1.0,0.0,0.0


In [45]:
# export this df_index to excel format for reference
df_index.to_excel(f"{store_folder}/reference_Part2.1_df_useful_cols_with_conversion.xlsx", na_rep='N.A.')

##Part 2.2: assign weighting score to each sample according to city group and industry

In [46]:
# mapping City, City_group, Industry in dataset
city_industry = df[['Q01','Q02']].copy()
city_industry['City']=city_industry['Q01']
city_industry = city_industry.replace({'City': City_dict})

city_industry['City_group'] = city_industry['Q01'].replace(City_group_dict)

city_industry['Industry'] = city_industry['Q02'].replace(Industry_dict)
# just have a look on top 10 rows
city_industry.head(10)

,Q01,Q02,City,City_group,Industry
0,11,4,Zhaoqing,Macao and other GBA cities,Professional Services
1,9,4,Jiangmen,Macao and other GBA cities,Professional Services
2,10,3,Zhuhai,Macao and other GBA cities,Financial Services
3,4,5,Shenzhen,Shenzhen,Innovation & Technology
4,9,3,Jiangmen,Macao and other GBA cities,Financial Services
5,9,3,Jiangmen,Macao and other GBA cities,Financial Services
6,1,3,Hong Kong,Hong Kong,Financial Services
7,6,2,Dongguan,Dongguan,Retail & Wholesale
8,4,5,Shenzhen,Shenzhen,Innovation & Technology
9,11,4,Zhaoqing,Macao and other GBA cities,Professional Services


In [47]:
for index, row in city_industry.iterrows():
    # assign the weighting score one by one from the WEIGHTED_DF from Part 1.3
    city_industry.at[index, 'weighting_score'] = WEIGHTED_DF.loc[row[3],row[4]]

In [48]:
city_industry.head(10)

,Q01,Q02,City,City_group,Industry,weighting_score
0,11,4,Zhaoqing,Macao and other GBA cities,Professional Services,0.073213
1,9,4,Jiangmen,Macao and other GBA cities,Professional Services,0.073213
2,10,3,Zhuhai,Macao and other GBA cities,Financial Services,0.022259
3,4,5,Shenzhen,Shenzhen,Innovation & Technology,11.087420
4,9,3,Jiangmen,Macao and other GBA cities,Financial Services,0.022259
5,9,3,Jiangmen,Macao and other GBA cities,Financial Services,0.022259
6,1,3,Hong Kong,Hong Kong,Financial Services,10.101462
7,6,2,Dongguan,Dongguan,Retail & Wholesale,2.043363
8,4,5,Shenzhen,Shenzhen,Innovation & Technology,11.087420
9,11,4,Zhaoqing,Macao and other GBA cities,Professional Services,0.073213


In [49]:
city_industry['weighting_score']

0       0.073213
1       0.073213
2       0.022259
3      11.087420
4       0.022259
         ...    
995     0.073213
996    10.101462
997    18.747256
998     0.073213
999     0.073213
Name: weighting_score, Length: 1000, dtype: float64

In [50]:
# multiply the dataframe column Q03-10 A and B, 13-17A by the above weighting_score column
DF_INDEX_WEIGHTED = df_converted.multiply(city_industry['weighting_score'], axis="index")

In [51]:
DF_INDEX_WEIGHTED

,Q03A,Q04A,Q05A,Q06A,Q07A,Q08A,Q09A,Q10A,Q11A,Q12A,...,Q08B,Q09B,Q10B,Q11B,Q12B,Q13B,Q14B,Q15B,Q16B,Q17B
0,-0.073213,0.000000,0.000000,0.073213,0.000000,0.000000,0.000000,0.073213,0.073213,0.000000,...,0.073213,NaN,0.073213,NaN,-0.073213,-0.073213,0.073213,0.073213,-0.073213,0.000000
1,0.000000,0.073213,-0.073213,0.073213,-0.073213,-0.073213,-0.073213,0.073213,0.000000,0.000000,...,0.073213,0.000000,-0.073213,-0.073213,NaN,0.073213,0.000000,-0.073213,0.073213,0.073213
2,-0.022259,0.022259,0.022259,0.022259,-0.022259,0.022259,0.022259,0.000000,0.022259,NaN,...,0.000000,-0.022259,0.022259,0.022259,0.022259,0.022259,0.022259,0.000000,0.000000,0.022259
3,11.087420,0.000000,0.000000,NaN,11.087420,-11.087420,0.000000,-11.087420,0.000000,0.000000,...,11.087420,11.087420,-11.087420,11.087420,11.087420,-11.087420,-11.087420,11.087420,11.087420,-11.087420
4,0.022259,0.000000,0.000000,-0.022259,0.000000,0.022259,0.022259,0.022259,-0.022259,NaN,...,-0.022259,0.000000,0.022259,-0.022259,0.000000,-0.022259,-0.022259,-0.022259,0.022259,0.022259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.073213,0.073213,0.073213,0.073213,0.073213,0.073213,NaN,0.000000,0.000000,0.073213,...,0.073213,0.073213,-0.073213,NaN,-0.073213,0.073213,-0.073213,-0.073213,0.073213,0.000000
996,10.101462,-10.101462,0.000000,NaN,-10.101462,10.101462,0.000000,0.000000,NaN,10.101462,...,-10.101462,NaN,10.101462,10.101462,-10.101462,10.101462,0.000000,10.101462,-10.101462,-10.101462
997,0.000000,-18.747256,-18.747256,18.747256,18.747256,-18.747256,0.000000,-18.747256,18.747256,NaN,...,-18.747256,NaN,18.747256,18.747256,18.747256,0.000000,18.747256,-18.747256,0.000000,-18.747256
998,0.000000,0.073213,0.073213,0.000000,0.000000,-0.073213,0.073213,0.073213,0.000000,0.000000,...,-0.073213,NaN,0.073213,0.000000,0.073213,-0.073213,0.000000,0.073213,0.073213,-0.073213


In [52]:
# merge 2 dataframe
DF_INDEX_WEIGHTED = pd.concat([city_industry, DF_INDEX_WEIGHTED], sort=False, axis=1)

In [53]:
DF_INDEX_WEIGHTED.head(10)

,Q01,Q02,City,City_group,Industry,weighting_score,Q03A,Q04A,Q05A,Q06A,...,Q08B,Q09B,Q10B,Q11B,Q12B,Q13B,Q14B,Q15B,Q16B,Q17B
0,11,4,Zhaoqing,Macao and other GBA cities,Professional Services,0.073213,-0.073213,0.000000,0.000000,0.073213,...,0.073213,NaN,0.073213,NaN,-0.073213,-0.073213,0.073213,0.073213,-0.073213,0.000000
1,9,4,Jiangmen,Macao and other GBA cities,Professional Services,0.073213,0.000000,0.073213,-0.073213,0.073213,...,0.073213,0.000000,-0.073213,-0.073213,NaN,0.073213,0.000000,-0.073213,0.073213,0.073213
2,10,3,Zhuhai,Macao and other GBA cities,Financial Services,0.022259,-0.022259,0.022259,0.022259,0.022259,...,0.000000,-0.022259,0.022259,0.022259,0.022259,0.022259,0.022259,0.000000,0.000000,0.022259
3,4,5,Shenzhen,Shenzhen,Innovation & Technology,11.087420,11.087420,0.000000,0.000000,NaN,...,11.087420,11.087420,-11.087420,11.087420,11.087420,-11.087420,-11.087420,11.087420,11.087420,-11.087420
4,9,3,Jiangmen,Macao and other GBA cities,Financial Services,0.022259,0.022259,0.000000,0.000000,-0.022259,...,-0.022259,0.000000,0.022259,-0.022259,0.000000,-0.022259,-0.022259,-0.022259,0.022259,0.022259
5,9,3,Jiangmen,Macao and other GBA cities,Financial Services,0.022259,0.000000,-0.022259,-0.022259,-0.022259,...,0.022259,0.000000,0.000000,0.022259,-0.022259,-0.022259,-0.022259,0.022259,0.000000,0.022259
6,1,3,Hong Kong,Hong Kong,Financial Services,10.101462,10.101462,0.000000,10.101462,10.101462,...,10.101462,0.000000,10.101462,10.101462,NaN,0.000000,0.000000,10.101462,-10.101462,10.101462
7,6,2,Dongguan,Dongguan,Retail & Wholesale,2.043363,2.043363,0.000000,-2.043363,-2.043363,...,2.043363,-2.043363,-2.043363,2.043363,NaN,0.000000,-2.043363,2.043363,-2.043363,0.000000
8,4,5,Shenzhen,Shenzhen,Innovation & Technology,11.087420,11.087420,0.000000,0.000000,0.000000,...,-11.087420,0.000000,-11.087420,11.087420,11.087420,11.087420,-11.087420,11.087420,0.000000,11.087420
9,11,4,Zhaoqing,Macao and other GBA cities,Professional Services,0.073213,0.000000,-0.073213,0.000000,0.000000,...,0.000000,-0.073213,0.073213,0.073213,0.073213,-0.073213,0.073213,0.073213,0.000000,0.000000


In [54]:
# export this DF_INDEX_WEIGHTED to excel format for reference
DF_INDEX_WEIGHTED.to_excel(f"{store_folder}/reference_Part2.2_DF_INDEX_WEIGHTED.xlsx", na_rep='N.A.')

##Part 2.3: Weighted overall index for Headline, Performance, Expectation, Credit

###Part 2.3.1: responses_weighted_sum_byquestion, weighted_index_byquestion

In [55]:
# calculate each column responses weighting sum
responses_weighted_sum_byquestion = DF_INDEX_WEIGHTED[calculation_columns].sum()
responses_weighted_sum_byquestion

Q03A    -224.748591
Q04A     289.105608
Q05A     102.760394
Q06A   -2341.975022
Q07A    -105.150961
Q08A    -488.082746
Q09A   -1630.220619
Q10A   -4457.105905
Q11A    -658.109968
Q12A   -2400.104827
Q13A     596.970683
Q14A    2334.418700
Q15A   -1068.127210
Q16A    1931.264027
Q17A    -282.705347
Q03B    1910.635730
Q04B    -402.437416
Q05B    1105.225631
Q06B      79.520349
Q07B    1585.716197
Q08B    1654.205070
Q09B     996.157844
Q10B     195.089530
Q11B     673.803066
Q12B   -1070.904804
Q13B     275.136675
Q14B    1012.913416
Q15B   -2851.878766
Q16B   -1339.258818
Q17B    2718.643331
dtype: float64

In [56]:
weighted_index_byquestion = (responses_weighted_sum_byquestion / WEIGHTED_TOTAL_SUM * 100 + 100)/2
weighted_index_byquestion

Q03A    49.365994
Q04A    50.815554
Q05A    50.289883
Q06A    43.393390
Q07A    49.703374
Q08A    48.623140
Q09A    45.401219
Q10A    37.426698
Q11A    48.143500
Q12A    43.229409
Q13A    51.684028
Q14A    56.585293
Q15A    46.986860
Q16A    55.448012
Q17A    49.202501
Q03B    55.389820
Q04B    48.864742
Q05B    53.117793
Q06B    50.224323
Q07B    54.473236
Q08B    54.666440
Q09B    52.810118
Q10B    50.550339
Q11B    51.900769
Q12B    46.979024
Q13B    50.776149
Q14B    52.857385
Q15B    41.954974
Q16B    46.222009
Q17B    57.669174
dtype: float64

###Part 2.3.2: Weighted overall index for Performance sub-index

In [57]:
# Columns for Performance sub-index
weighted_index_byquestion[performance_columns]

Q03A    49.365994
Q04A    50.815554
Q05A    50.289883
Q06A    43.393390
Q07A    49.703374
Q08A    48.623140
Q09A    45.401219
Q10A    37.426698
dtype: float64

In [58]:
WEIGHTED_OVERALL_PERFORMANCE_SUB_INDEX = weighted_index_byquestion[performance_columns].mean()
WEIGHTED_OVERALL_PERFORMANCE_SUB_INDEX

46.877406504198774

###Part 2.3.3: Weighted overall index for Expectation sub-index

In [59]:
# Columns for Expectation sub-index
weighted_index_byquestion[expectation_columns]

Q03B    55.389820
Q04B    48.864742
Q05B    53.117793
Q06B    50.224323
Q07B    54.473236
Q08B    54.666440
Q09B    52.810118
Q10B    50.550339
dtype: float64

In [60]:
WEIGHTED_OVERALL_EXPECTATION_SUB_INDEX = weighted_index_byquestion[expectation_columns].mean()
WEIGHTED_OVERALL_EXPECTATION_SUB_INDEX

52.51210153054471

###Part 2.3.4: Weighted overall index for Credit sub-index

In [61]:
# Columns for Credit sub-index
weighted_index_byquestion[credit_columns]

Q13A    51.684028
Q14A    56.585293
Q15A    46.986860
Q16A    55.448012
Q17A    49.202501
dtype: float64

In [62]:
# Q13, Q14 are related to financing cost, need to adjusted to 100-X
weighted_index_byquestion['Q13A']= 100 - weighted_index_byquestion['Q13A']
weighted_index_byquestion['Q14A']= 100 - weighted_index_byquestion['Q14A']

In [63]:
weighted_index_byquestion[credit_columns]

Q13A    48.315972
Q14A    43.414707
Q15A    46.986860
Q16A    55.448012
Q17A    49.202501
dtype: float64

In [64]:
WEIGHTED_OVERALL_CREDIT_SUB_INDEX = weighted_index_byquestion[credit_columns].mean()
WEIGHTED_OVERALL_CREDIT_SUB_INDEX

48.673610027471035

###Part 2.3.5: Weighted overall Headline Index

In [65]:
WEIGHTED_OVERALL_HEADLINE_INDEX = np.mean([WEIGHTED_OVERALL_PERFORMANCE_SUB_INDEX,
                                         WEIGHTED_OVERALL_EXPECTATION_SUB_INDEX,
                                         WEIGHTED_OVERALL_CREDIT_SUB_INDEX])
WEIGHTED_OVERALL_HEADLINE_INDEX

49.35437268740484

###Part 2.3.6: Store the Indices to dataframe and export to Excel

In [66]:
WEIGHTED_INDEX_BYQUESTION_DF = pd.DataFrame({"responses_weighted_sum_byquestion": responses_weighted_sum_byquestion,
                                             "WEIGHTED_TOTAL_SUM":WEIGHTED_TOTAL_SUM,
                                             "weighted_index_byquestion": weighted_index_byquestion})
WEIGHTED_INDEX_BYQUESTION_DF

,responses_weighted_sum_byquestion,WEIGHTED_TOTAL_SUM,weighted_index_byquestion
Q03A,-224.748591,17724.484976,49.365994
Q04A,289.105608,17724.484976,50.815554
Q05A,102.760394,17724.484976,50.289883
Q06A,-2341.975022,17724.484976,43.393390
Q07A,-105.150961,17724.484976,49.703374
Q08A,-488.082746,17724.484976,48.623140
Q09A,-1630.220619,17724.484976,45.401219
Q10A,-4457.105905,17724.484976,37.426698
Q11A,-658.109968,17724.484976,48.143500
Q12A,-2400.104827,17724.484976,43.229409


In [67]:
WEIGHTED_INDEX_OVERALL_DF = pd.DataFrame({'WEIGHTED_OVERALL_HEADLINE_INDEX' : WEIGHTED_OVERALL_HEADLINE_INDEX,
                            'WEIGHTED_OVERALL_PERFORMANCE_SUB_INDEX': WEIGHTED_OVERALL_PERFORMANCE_SUB_INDEX,
                            'WEIGHTED_OVERALL_EXPECTATION_SUB_INDEX': WEIGHTED_OVERALL_EXPECTATION_SUB_INDEX,
                            'WEIGHTED_OVERALL_CREDIT_SUB_INDEX': WEIGHTED_OVERALL_CREDIT_SUB_INDEX},
                            index=[0])    

WEIGHTED_INDEX_OVERALL_DF              

,WEIGHTED_OVERALL_HEADLINE_INDEX,WEIGHTED_OVERALL_PERFORMANCE_SUB_INDEX,WEIGHTED_OVERALL_EXPECTATION_SUB_INDEX,WEIGHTED_OVERALL_CREDIT_SUB_INDEX
0,49.354373,46.877407,52.512102,48.67361


In [68]:
# export excel file
with pd.ExcelWriter(f'{store_folder}/reference_Part2.3.6_WEIGHTED_INDEX_OVERALL.xlsx') as writer:  
    WEIGHTED_INDEX_BYQUESTION_DF.to_excel(writer, na_rep='N.A.',
                               sheet_name='WEIGHTED_INDEX_BYQUESTION')
    WEIGHTED_INDEX_OVERALL_DF.to_excel(writer, sheet_name='WEIGHTED_INDEX_OVERALL',index=False)

#Part 3: Compile weighted-index by Industry

In [69]:
# have a look first on part 1.4
WEIGHTED_SUM_score

,Manufacturing & Trading,Retail & Wholesale,Financial Services,Professional Services,Innovation & Technology
Hong Kong,2157.68,512.484,232.334,371.657,22.8198
Guangzhou,2072.84,621.374,16.1453,38.3634,27.8681
Shenzhen,5824.69,777.172,35.25,24.3364,243.923
Foshan,1132.45,25.5482,0.506747,0.768006,NaN
Dongguan,1210.36,46.9973,0.532842,2.69479,NaN
Macao and other GBA cities,2212.18,102.525,2.78239,8.19989,NaN


##Part 3.1: Weighted sum by industry

In [70]:
WEIGHTED_SUM_BY_INDUSTRY = WEIGHTED_SUM_score.sum()
WEIGHTED_SUM_BY_INDUSTRY.columns=["WEIGHTED_SUM_BYINDUSTRY"]
WEIGHTED_SUM_BY_INDUSTRY_DF=pd.DataFrame(WEIGHTED_SUM_BY_INDUSTRY,columns=["WEIGHTED_SUM_BY_INDUSTRY"]).T
WEIGHTED_SUM_BY_INDUSTRY_DF

,Manufacturing & Trading,Retail & Wholesale,Financial Services,Professional Services,Innovation & Technology
WEIGHTED_SUM_BY_INDUSTRY,14610.203275,2086.100142,287.550897,446.019542,294.611119


In [71]:
# export WEIGHTED_SUM_BYINDUSTRY
WEIGHTED_SUM_BY_INDUSTRY_DF.to_excel(f"{store_folder}/reference_Part3.1_WEIGHTED_SUM_BY_INDUSTRY.xlsx", \
                                                                                 na_rep='N.A.')

In [72]:
# just have a look first
DF_INDEX_WEIGHTED.head(10)

,Q01,Q02,City,City_group,Industry,weighting_score,Q03A,Q04A,Q05A,Q06A,...,Q08B,Q09B,Q10B,Q11B,Q12B,Q13B,Q14B,Q15B,Q16B,Q17B
0,11,4,Zhaoqing,Macao and other GBA cities,Professional Services,0.073213,-0.073213,0.000000,0.000000,0.073213,...,0.073213,NaN,0.073213,NaN,-0.073213,-0.073213,0.073213,0.073213,-0.073213,0.000000
1,9,4,Jiangmen,Macao and other GBA cities,Professional Services,0.073213,0.000000,0.073213,-0.073213,0.073213,...,0.073213,0.000000,-0.073213,-0.073213,NaN,0.073213,0.000000,-0.073213,0.073213,0.073213
2,10,3,Zhuhai,Macao and other GBA cities,Financial Services,0.022259,-0.022259,0.022259,0.022259,0.022259,...,0.000000,-0.022259,0.022259,0.022259,0.022259,0.022259,0.022259,0.000000,0.000000,0.022259
3,4,5,Shenzhen,Shenzhen,Innovation & Technology,11.087420,11.087420,0.000000,0.000000,NaN,...,11.087420,11.087420,-11.087420,11.087420,11.087420,-11.087420,-11.087420,11.087420,11.087420,-11.087420
4,9,3,Jiangmen,Macao and other GBA cities,Financial Services,0.022259,0.022259,0.000000,0.000000,-0.022259,...,-0.022259,0.000000,0.022259,-0.022259,0.000000,-0.022259,-0.022259,-0.022259,0.022259,0.022259
5,9,3,Jiangmen,Macao and other GBA cities,Financial Services,0.022259,0.000000,-0.022259,-0.022259,-0.022259,...,0.022259,0.000000,0.000000,0.022259,-0.022259,-0.022259,-0.022259,0.022259,0.000000,0.022259
6,1,3,Hong Kong,Hong Kong,Financial Services,10.101462,10.101462,0.000000,10.101462,10.101462,...,10.101462,0.000000,10.101462,10.101462,NaN,0.000000,0.000000,10.101462,-10.101462,10.101462
7,6,2,Dongguan,Dongguan,Retail & Wholesale,2.043363,2.043363,0.000000,-2.043363,-2.043363,...,2.043363,-2.043363,-2.043363,2.043363,NaN,0.000000,-2.043363,2.043363,-2.043363,0.000000
8,4,5,Shenzhen,Shenzhen,Innovation & Technology,11.087420,11.087420,0.000000,0.000000,0.000000,...,-11.087420,0.000000,-11.087420,11.087420,11.087420,11.087420,-11.087420,11.087420,0.000000,11.087420
9,11,4,Zhaoqing,Macao and other GBA cities,Professional Services,0.073213,0.000000,-0.073213,0.000000,0.000000,...,0.000000,-0.073213,0.073213,0.073213,0.073213,-0.073213,0.073213,0.073213,0.000000,0.000000


In [73]:
# just have a look for selecting one industry "Manufacturing & Trading"
DF_INDEX_WEIGHTED[DF_INDEX_WEIGHTED['Q02']==1]

,Q01,Q02,City,City_group,Industry,weighting_score,Q03A,Q04A,Q05A,Q06A,...,Q08B,Q09B,Q10B,Q11B,Q12B,Q13B,Q14B,Q15B,Q16B,Q17B
11,10,1,Zhuhai,Macao and other GBA cities,Manufacturing & Trading,18.747256,18.747256,18.747256,0.000000,0.000000,...,18.747256,-18.747256,18.747256,18.747256,18.747256,-18.747256,0.000000,18.747256,0.000000,0.000000
14,8,1,Zhongshan,Macao and other GBA cities,Manufacturing & Trading,18.747256,18.747256,0.000000,-18.747256,NaN,...,0.000000,0.000000,18.747256,0.000000,NaN,0.000000,18.747256,18.747256,18.747256,-18.747256
15,7,1,Huizhou,Macao and other GBA cities,Manufacturing & Trading,18.747256,18.747256,18.747256,-18.747256,-18.747256,...,18.747256,-18.747256,18.747256,NaN,0.000000,0.000000,18.747256,-18.747256,18.747256,-18.747256
18,8,1,Zhongshan,Macao and other GBA cities,Manufacturing & Trading,18.747256,-18.747256,-18.747256,-18.747256,NaN,...,18.747256,0.000000,-18.747256,0.000000,0.000000,-18.747256,-18.747256,18.747256,-18.747256,0.000000
27,4,1,Shenzhen,Shenzhen,Manufacturing & Trading,448.053431,-448.053431,0.000000,0.000000,NaN,...,448.053431,0.000000,-448.053431,448.053431,NaN,448.053431,448.053431,-448.053431,448.053431,448.053431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,2,1,Macao,Macao and other GBA cities,Manufacturing & Trading,18.747256,-18.747256,-18.747256,0.000000,NaN,...,18.747256,0.000000,18.747256,NaN,NaN,0.000000,-18.747256,-18.747256,0.000000,0.000000
984,2,1,Macao,Macao and other GBA cities,Manufacturing & Trading,18.747256,-18.747256,0.000000,18.747256,18.747256,...,-18.747256,-18.747256,0.000000,-18.747256,0.000000,18.747256,0.000000,0.000000,18.747256,18.747256
985,9,1,Jiangmen,Macao and other GBA cities,Manufacturing & Trading,18.747256,0.000000,18.747256,18.747256,-18.747256,...,18.747256,18.747256,0.000000,0.000000,-18.747256,18.747256,18.747256,0.000000,-18.747256,0.000000
991,6,1,Dongguan,Dongguan,Manufacturing & Trading,75.647700,0.000000,-75.647700,75.647700,0.000000,...,75.647700,NaN,-75.647700,-75.647700,75.647700,75.647700,0.000000,75.647700,75.647700,-75.647700


In [74]:
# acquire column names
industry_names = WEIGHTED_SUM_score.columns
selected_columns_list= performance_columns+expectation_columns+credit_columns
industry_names

Index(['Manufacturing & Trading', 'Retail & Wholesale', 'Financial Services',
       'Professional Services', 'Innovation & Technology'],
      dtype='object')

##Part 3.2: responses_weighted_sum_byquestion_industry

In [75]:
# calculate each column weighting sum by industry
responses_weighted_sum_byquestion_industry = DF_INDEX_WEIGHTED.groupby(['Industry']).sum()[calculation_columns].T[industry_names]
responses_weighted_sum_byquestion_industry

Industry,Manufacturing & Trading,Retail & Wholesale,Financial Services,Professional Services,Innovation & Technology
Q03A,-166.374351,-143.077208,-20.846496,40.878470,6.467099e+01
Q04A,145.838919,-53.490764,39.615007,65.261166,9.188128e+01
Q05A,5.536858,95.782771,47.056326,47.113328,-9.272889e+01
Q06A,-2195.557865,-287.069145,42.231019,43.844299,5.457667e+01
Q07A,-404.408095,302.614571,-18.686702,-56.308749,7.163801e+01
Q08A,-549.902450,61.153448,-14.937639,-67.121538,8.272543e+01
Q09A,-1703.377888,54.791201,6.086477,1.303940,1.097565e+01
Q10A,-3793.103223,-531.632885,-31.921156,-20.458058,-7.999058e+01
Q11A,-491.494911,-197.266658,-17.980616,-23.117565,7.174978e+01
Q12A,-2675.630855,231.736295,12.710817,66.306467,-3.522755e+01


In [76]:
# export responses_weighted_sum_byquestion_industry
responses_weighted_sum_byquestion_industry.to_excel(f"{store_folder}/reference_Part3.2_responses_weighted_sum_byquestion_industry.xlsx", na_rep='N.A.')

##Part 3.3: WEIGHTED_INDEX_BYQUESTION_INDUSTRY

In [77]:
# have a look again
WEIGHTED_SUM_BY_INDUSTRY_DF.values

array([[14610.20327534,  2086.10014236,   287.55089695,   446.01954239,
          294.61111886]])

In [78]:
# divide each column by according industry weighted sum value(WEIGHTED_SUM_BYINDUSTRY)
responses_weighted_sum_byquestion_industry / WEIGHTED_SUM_BY_INDUSTRY

Industry,Manufacturing & Trading,Retail & Wholesale,Financial Services,Professional Services,Innovation & Technology
Q03A,-0.011388,-0.068586,-0.072497,0.091652,2.195131e-01
Q04A,0.009982,-0.025642,0.137767,0.146319,3.118731e-01
Q05A,0.000379,0.045915,0.163645,0.105631,-3.147501e-01
Q06A,-0.150276,-0.137610,0.146865,0.098301,1.852499e-01
Q07A,-0.027680,0.145062,-0.064986,-0.126247,2.431613e-01
Q08A,-0.037638,0.029315,-0.051948,-0.150490,2.807954e-01
Q09A,-0.116588,0.026265,0.021167,0.002924,3.725471e-02
Q10A,-0.259620,-0.254845,-0.111010,-0.045868,-2.715124e-01
Q11A,-0.033641,-0.094562,-0.062530,-0.051831,2.435406e-01
Q12A,-0.183134,0.111086,0.044204,0.148663,-1.195731e-01


In [79]:
WEIGHTED_INDEX_BYQUESTION_INDUSTRY = (responses_weighted_sum_byquestion_industry / WEIGHTED_SUM_BY_INDUSTRY * 100 + 100)/2
# export to excel for reference
#WEIGHTED_INDEX_BYQUESTION_INDUSTRY.to_excel(f"{store_folder}/reference_Part3.3_WEIGHTED_INDEX_BYQUESTION_INDUSTRY.xlsx", na_rep='N.A.')
WEIGHTED_INDEX_BYQUESTION_INDUSTRY

Industry,Manufacturing & Trading,Retail & Wholesale,Financial Services,Professional Services,Innovation & Technology
Q03A,49.430623,46.570701,46.375164,54.582587,60.975654
Q04A,50.499100,48.717924,56.888347,57.315954,65.593655
Q05A,50.018949,52.295738,58.182260,55.281532,34.262493
Q06A,42.486217,43.119478,57.343225,54.915065,59.262493
Q07A,48.616008,57.253117,46.750714,43.687637,62.158063
Q08A,48.118088,51.465736,47.402610,42.475494,64.039768
Q09A,44.170588,51.313245,51.058330,50.146175,51.862735
Q10A,37.018992,37.257733,44.449477,47.706596,36.424378
Q11A,48.317974,45.271879,46.873490,47.408458,62.177032
Q12A,40.843280,55.554295,52.210186,57.433135,44.021347


In [80]:
# Q13, Q14 are related to financing cost, need to adjusted to 100-X
WEIGHTED_INDEX_BYQUESTION_INDUSTRY.loc['Q13A'] = 100-WEIGHTED_INDEX_BYQUESTION_INDUSTRY.loc['Q13A']
WEIGHTED_INDEX_BYQUESTION_INDUSTRY.loc['Q14A'] = 100-WEIGHTED_INDEX_BYQUESTION_INDUSTRY.loc['Q14A']
WEIGHTED_INDEX_BYQUESTION_INDUSTRY

Industry,Manufacturing & Trading,Retail & Wholesale,Financial Services,Professional Services,Innovation & Technology
Q03A,49.430623,46.570701,46.375164,54.582587,60.975654
Q04A,50.499100,48.717924,56.888347,57.315954,65.593655
Q05A,50.018949,52.295738,58.182260,55.281532,34.262493
Q06A,42.486217,43.119478,57.343225,54.915065,59.262493
Q07A,48.616008,57.253117,46.750714,43.687637,62.158063
Q08A,48.118088,51.465736,47.402610,42.475494,64.039768
Q09A,44.170588,51.313245,51.058330,50.146175,51.862735
Q10A,37.018992,37.257733,44.449477,47.706596,36.424378
Q11A,48.317974,45.271879,46.873490,47.408458,62.177032
Q12A,40.843280,55.554295,52.210186,57.433135,44.021347


In [81]:
# export excel file
with pd.ExcelWriter(f"{store_folder}/reference_Part3.3_WEIGHTED_INDEX_BYQUESTION_INDUSTRY.xlsx") as writer:  
    responses_weighted_sum_byquestion_industry.to_excel(writer, na_rep='N.A.', sheet_name='responses_w_sum_byquestion_ind')
    WEIGHTED_SUM_BY_INDUSTRY_DF.to_excel(writer, sheet_name='WEIGHTED_SUM_BY_INDUSTRY')
    WEIGHTED_INDEX_BYQUESTION_INDUSTRY.to_excel(writer, sheet_name='WEIGHTED_INDEX_BYQ_INDUSTRY')

##Part 3.4: WEIGHTED_BY_INDUSTRY_INDEX

In [82]:
# Weighted performance index by industry
WEIGHTED_INDEX_BYQUESTION_INDUSTRY.loc[performance_columns]

Industry,Manufacturing & Trading,Retail & Wholesale,Financial Services,Professional Services,Innovation & Technology
Q03A,49.430623,46.570701,46.375164,54.582587,60.975654
Q04A,50.499100,48.717924,56.888347,57.315954,65.593655
Q05A,50.018949,52.295738,58.182260,55.281532,34.262493
Q06A,42.486217,43.119478,57.343225,54.915065,59.262493
Q07A,48.616008,57.253117,46.750714,43.687637,62.158063
Q08A,48.118088,51.465736,47.402610,42.475494,64.039768
Q09A,44.170588,51.313245,51.058330,50.146175,51.862735
Q10A,37.018992,37.257733,44.449477,47.706596,36.424378


In [83]:
WEIGHTED_BY_INDUSTRY_PERFORMANCE_SUB_INDEX = WEIGHTED_INDEX_BYQUESTION_INDUSTRY.loc[performance_columns].mean()
WEIGHTED_BY_INDUSTRY_PERFORMANCE_SUB_INDEX

Industry
Manufacturing & Trading    46.294821
Retail & Wholesale         48.499209
Financial Services         51.056266
Professional Services      50.763880
Innovation & Technology    54.322405
dtype: float64

In [84]:
WEIGHTED_BY_INDUSTRY_EXPECTATION_SUB_INDEX = WEIGHTED_INDEX_BYQUESTION_INDUSTRY.loc[expectation_columns].mean()
WEIGHTED_BY_INDUSTRY_EXPECTATION_SUB_INDEX

Industry
Manufacturing & Trading    52.889884
Retail & Wholesale         51.067955
Financial Services         52.625850
Professional Services      47.909947
Innovation & Technology    50.859395
dtype: float64

In [85]:
WEIGHTED_BY_INDUSTRY_CREDIT_SUB_INDEX = WEIGHTED_INDEX_BYQUESTION_INDUSTRY.loc[credit_columns].mean()
WEIGHTED_BY_INDUSTRY_CREDIT_SUB_INDEX

Industry
Manufacturing & Trading    48.831147
Retail & Wholesale         47.300098
Financial Services         47.691894
Professional Services      50.354699
Innovation & Technology    48.999890
dtype: float64

In [86]:
WEIGHTED_BY_INDUSTRY_INDEX_DF = pd.DataFrame({
                                            'WEIGHTED_BY_INDUSTRY_PERFORMANCE_SUB_INDEX': WEIGHTED_BY_INDUSTRY_PERFORMANCE_SUB_INDEX,
                                            'WEIGHTED_BY_INDUSTRY_EXPECTATION_SUB_INDEX': WEIGHTED_BY_INDUSTRY_EXPECTATION_SUB_INDEX,
                                            'WEIGHTED_BY_INDUSTRY_CREDIT_SUB_INDEX': WEIGHTED_BY_INDUSTRY_CREDIT_SUB_INDEX}
                                            ).T    

WEIGHTED_BY_INDUSTRY_INDEX_DF      

Industry,Manufacturing & Trading,Retail & Wholesale,Financial Services,Professional Services,Innovation & Technology
WEIGHTED_BY_INDUSTRY_PERFORMANCE_SUB_INDEX,46.294821,48.499209,51.056266,50.763880,54.322405
WEIGHTED_BY_INDUSTRY_EXPECTATION_SUB_INDEX,52.889884,51.067955,52.625850,47.909947,50.859395
WEIGHTED_BY_INDUSTRY_CREDIT_SUB_INDEX,48.831147,47.300098,47.691894,50.354699,48.999890


In [87]:
WEIGHTED_BY_INDUSTRY_INDEX_DF.loc['WEIGHTED_BY_INDUSTRY_HEADLINE_INDEX'] = WEIGHTED_BY_INDUSTRY_INDEX_DF.mean() 
WEIGHTED_BY_INDUSTRY_INDEX_DF

Industry,Manufacturing & Trading,Retail & Wholesale,Financial Services,Professional Services,Innovation & Technology
WEIGHTED_BY_INDUSTRY_PERFORMANCE_SUB_INDEX,46.294821,48.499209,51.056266,50.763880,54.322405
WEIGHTED_BY_INDUSTRY_EXPECTATION_SUB_INDEX,52.889884,51.067955,52.625850,47.909947,50.859395
WEIGHTED_BY_INDUSTRY_CREDIT_SUB_INDEX,48.831147,47.300098,47.691894,50.354699,48.999890
WEIGHTED_BY_INDUSTRY_HEADLINE_INDEX,49.338617,48.955754,50.458003,49.676175,51.393897


In [88]:
# export WEIGHTED_BY_INDUSTRY_INDEX result
WEIGHTED_BY_INDUSTRY_INDEX_DF.to_excel(f"{store_folder}/reference_Part3.4_WEIGHTED_BY_INDUSTRY_INDEX.xlsx", na_rep='N.A.')

#Part 4: Compile weighted-index by City group

In [89]:
# have a look first
WEIGHTED_SUM_score

,Manufacturing & Trading,Retail & Wholesale,Financial Services,Professional Services,Innovation & Technology
Hong Kong,2157.68,512.484,232.334,371.657,22.8198
Guangzhou,2072.84,621.374,16.1453,38.3634,27.8681
Shenzhen,5824.69,777.172,35.25,24.3364,243.923
Foshan,1132.45,25.5482,0.506747,0.768006,NaN
Dongguan,1210.36,46.9973,0.532842,2.69479,NaN
Macao and other GBA cities,2212.18,102.525,2.78239,8.19989,NaN


##Part 4.1: Weighted sum by city group

In [90]:
WEIGHTED_SUM_BY_CITY_GROUP = WEIGHTED_SUM_score.sum(axis=1)
WEIGHTED_SUM_BY_CITY_GROUP_DF = pd.DataFrame(WEIGHTED_SUM_BY_CITY_GROUP,columns=["WEIGHTED_SUM_BY_CITY_GROUP"]).T
WEIGHTED_SUM_BY_CITY_GROUP_DF


,Hong Kong,Guangzhou,Shenzhen,Foshan,Dongguan,Macao and other GBA cities
WEIGHTED_SUM_BY_CITY_GROUP,3296.972926,2776.588583,6905.376114,1159.276099,1260.588171,2325.683083


In [91]:
# export WEIGHTED_SUM_BY_CITY_GROUP
WEIGHTED_SUM_BY_CITY_GROUP_DF.to_excel(f"{store_folder}/reference_Part4.1_WEIGHTED_SUM_BY_CITY_GROUP.xlsx",\
                                                                           na_rep='N.A.')
                                                                                        

##Part 4.2: responses_weighted_sum_byquestion_city_group

In [92]:
# just have a look first
DF_INDEX_WEIGHTED.head(10)

,Q01,Q02,City,City_group,Industry,weighting_score,Q03A,Q04A,Q05A,Q06A,...,Q08B,Q09B,Q10B,Q11B,Q12B,Q13B,Q14B,Q15B,Q16B,Q17B
0,11,4,Zhaoqing,Macao and other GBA cities,Professional Services,0.073213,-0.073213,0.000000,0.000000,0.073213,...,0.073213,NaN,0.073213,NaN,-0.073213,-0.073213,0.073213,0.073213,-0.073213,0.000000
1,9,4,Jiangmen,Macao and other GBA cities,Professional Services,0.073213,0.000000,0.073213,-0.073213,0.073213,...,0.073213,0.000000,-0.073213,-0.073213,NaN,0.073213,0.000000,-0.073213,0.073213,0.073213
2,10,3,Zhuhai,Macao and other GBA cities,Financial Services,0.022259,-0.022259,0.022259,0.022259,0.022259,...,0.000000,-0.022259,0.022259,0.022259,0.022259,0.022259,0.022259,0.000000,0.000000,0.022259
3,4,5,Shenzhen,Shenzhen,Innovation & Technology,11.087420,11.087420,0.000000,0.000000,NaN,...,11.087420,11.087420,-11.087420,11.087420,11.087420,-11.087420,-11.087420,11.087420,11.087420,-11.087420
4,9,3,Jiangmen,Macao and other GBA cities,Financial Services,0.022259,0.022259,0.000000,0.000000,-0.022259,...,-0.022259,0.000000,0.022259,-0.022259,0.000000,-0.022259,-0.022259,-0.022259,0.022259,0.022259
5,9,3,Jiangmen,Macao and other GBA cities,Financial Services,0.022259,0.000000,-0.022259,-0.022259,-0.022259,...,0.022259,0.000000,0.000000,0.022259,-0.022259,-0.022259,-0.022259,0.022259,0.000000,0.022259
6,1,3,Hong Kong,Hong Kong,Financial Services,10.101462,10.101462,0.000000,10.101462,10.101462,...,10.101462,0.000000,10.101462,10.101462,NaN,0.000000,0.000000,10.101462,-10.101462,10.101462
7,6,2,Dongguan,Dongguan,Retail & Wholesale,2.043363,2.043363,0.000000,-2.043363,-2.043363,...,2.043363,-2.043363,-2.043363,2.043363,NaN,0.000000,-2.043363,2.043363,-2.043363,0.000000
8,4,5,Shenzhen,Shenzhen,Innovation & Technology,11.087420,11.087420,0.000000,0.000000,0.000000,...,-11.087420,0.000000,-11.087420,11.087420,11.087420,11.087420,-11.087420,11.087420,0.000000,11.087420
9,11,4,Zhaoqing,Macao and other GBA cities,Professional Services,0.073213,0.000000,-0.073213,0.000000,0.000000,...,0.000000,-0.073213,0.073213,0.073213,0.073213,-0.073213,0.073213,0.073213,0.000000,0.000000


In [93]:
responses_weighted_sum_byquestion_city_GROUP = DF_INDEX_WEIGHTED.groupby(['City_group']).sum()[selected_columns_list].T[City_group_rows] # measure column order is correct
responses_weighted_sum_byquestion_city_GROUP

City_group,Hong Kong,Guangzhou,Shenzhen,Foshan,Dongguan,Macao and other GBA cities
Q03A,445.555671,306.306309,-969.090275,-208.644490,-74.216790,275.340984
Q04A,-112.229703,52.167318,177.174322,-218.016667,244.720437,145.289901
Q05A,501.088059,-470.595864,-337.556221,281.314089,56.767001,71.743331
Q06A,-143.562907,-439.393168,-1878.245819,139.595342,230.907334,-251.275804
Q07A,-171.397163,-540.175747,280.364706,135.823019,234.340619,-44.106394
Q08A,-798.893316,134.411967,-59.379212,-2.123405,146.799742,91.101479
Q09A,11.731448,-361.837236,-890.107790,-348.157993,-149.171007,107.321959
Q10A,-666.574508,14.870976,-3012.891377,-430.631772,-241.696085,-120.183140
Q03B,311.139913,449.086157,1453.206155,-210.405925,1.838896,-94.229466
Q04B,2.454337,-71.896245,21.026488,283.205363,-308.435864,-328.791495


In [94]:
# export responses_weighted_sum_byquestion_city_GROUP
responses_weighted_sum_byquestion_city_GROUP.to_excel(f"{store_folder}/reference_Part4.2_responses_weighted_sum_byquestion_city_GROUP.xlsx", na_rep='N.A.')

##Part 4.3: WEIGHTED_INDEX_BYQUESTION_CITY_GROUP

In [95]:
# divide each column by according city weighted sum value(WEIGHTED_SUM_BY_CITY)
responses_weighted_sum_byquestion_city_GROUP /WEIGHTED_SUM_BY_CITY_GROUP

City_group,Hong Kong,Guangzhou,Shenzhen,Foshan,Dongguan,Macao and other GBA cities
Q03A,0.135141,0.110317,-0.140339,-0.179978,-0.058875,0.118391
Q04A,-0.034040,0.018788,0.025657,-0.188063,0.194132,0.062472
Q05A,0.151984,-0.169487,-0.048883,0.242664,0.045032,0.030848
Q06A,-0.043544,-0.158249,-0.271998,0.120416,0.183174,-0.108044
Q07A,-0.051986,-0.194547,0.040601,0.117162,0.185898,-0.018965
Q08A,-0.242311,0.048409,-0.008599,-0.001832,0.116453,0.039172
Q09A,0.003558,-0.130317,-0.128901,-0.300324,-0.118334,0.046146
Q10A,-0.202178,0.005356,-0.436311,-0.371466,-0.191733,-0.051676
Q03B,0.094371,0.161740,0.210446,-0.181498,0.001459,-0.040517
Q04B,0.000744,-0.025894,0.003045,0.244295,-0.244676,-0.141374


In [96]:
WEIGHTED_INDEX_BYQUESTION_CITY_GROUP = (responses_weighted_sum_byquestion_city_GROUP /WEIGHTED_SUM_BY_CITY_GROUP * 100 + 100)/2
# export to excel for reference
WEIGHTED_INDEX_BYQUESTION_CITY_GROUP

City_group,Hong Kong,Guangzhou,Shenzhen,Foshan,Dongguan,Macao and other GBA cities
Q03A,56.757042,55.515875,42.983074,41.001087,47.056263,55.919572
Q04A,48.297989,50.939414,51.282872,40.596862,59.706597,53.123596
Q05A,57.599214,41.525646,47.555845,62.133179,52.251608,51.542414
Q06A,47.822807,42.087536,36.400119,56.020798,59.158714,44.597806
Q07A,47.400689,40.272672,52.030047,55.858096,59.294892,49.051754
Q08A,37.884442,52.420452,49.570051,49.908417,55.822669,51.958596
Q09A,50.177912,43.484140,43.554965,34.983819,44.083278,52.307321
Q10A,39.891113,50.267792,28.184451,31.426695,40.413361,47.416175
Q03B,54.718569,58.087013,60.522281,40.925116,50.072938,47.974155
Q04B,50.037221,48.705313,50.152247,62.214750,37.766192,42.931292


In [97]:
# Q13, Q14 are related to financing cost, need to adjusted to 100-X
WEIGHTED_INDEX_BYQUESTION_CITY_GROUP.loc['Q13A'] = 100-WEIGHTED_INDEX_BYQUESTION_CITY_GROUP.loc['Q13A']
WEIGHTED_INDEX_BYQUESTION_CITY_GROUP.loc['Q14A'] = 100-WEIGHTED_INDEX_BYQUESTION_CITY_GROUP.loc['Q14A']
WEIGHTED_INDEX_BYQUESTION_CITY_GROUP

City_group,Hong Kong,Guangzhou,Shenzhen,Foshan,Dongguan,Macao and other GBA cities
Q03A,56.757042,55.515875,42.983074,41.001087,47.056263,55.919572
Q04A,48.297989,50.939414,51.282872,40.596862,59.706597,53.123596
Q05A,57.599214,41.525646,47.555845,62.133179,52.251608,51.542414
Q06A,47.822807,42.087536,36.400119,56.020798,59.158714,44.597806
Q07A,47.400689,40.272672,52.030047,55.858096,59.294892,49.051754
Q08A,37.884442,52.420452,49.570051,49.908417,55.822669,51.958596
Q09A,50.177912,43.484140,43.554965,34.983819,44.083278,52.307321
Q10A,39.891113,50.267792,28.184451,31.426695,40.413361,47.416175
Q03B,54.718569,58.087013,60.522281,40.925116,50.072938,47.974155
Q04B,50.037221,48.705313,50.152247,62.214750,37.766192,42.931292


In [98]:
# export excel file
with pd.ExcelWriter(f"{store_folder}/reference_Part4.3_WEIGHTED_INDEX_BYQUESTION_CITY_GROUP.xlsx") as writer:  
    responses_weighted_sum_byquestion_city_GROUP.to_excel(writer, na_rep='N.A.', sheet_name='responses_w_sum_byq_ctygrp')
    WEIGHTED_SUM_BY_CITY_GROUP_DF.to_excel(writer, sheet_name='WEIGHTED_SUM_BY_CITY_GROUP')
    WEIGHTED_INDEX_BYQUESTION_CITY_GROUP.to_excel(writer, sheet_name='WEIGHTED_INDEX_BYQ_CITY_GROUP')

##Part 4.4: WEIGHTED_BY_CITY_GROUP_INDEX

In [99]:
WEIGHTED_BY_CITY_GROUP_PERFORMANCE_SUB_INDEX = WEIGHTED_INDEX_BYQUESTION_CITY_GROUP.loc[performance_columns].mean()

WEIGHTED_BY_CITY_GROUP_EXPECTATION_SUB_INDEX = WEIGHTED_INDEX_BYQUESTION_CITY_GROUP.loc[expectation_columns].mean()

WEIGHTED_BY_CITY_GROUP_CREDIT_SUB_INDEX = WEIGHTED_INDEX_BYQUESTION_CITY_GROUP.loc[credit_columns].mean()

WEIGHTED_BY_CITY_GROUP_INDEX_DF = pd.DataFrame({
                                            'WEIGHTED_BY_CITY_GROUP_PERFORMANCE_SUB_INDEX': WEIGHTED_BY_CITY_GROUP_PERFORMANCE_SUB_INDEX,
                                            'WEIGHTED_BY_CITY_GROUP_EXPECTATION_SUB_INDEX': WEIGHTED_BY_CITY_GROUP_EXPECTATION_SUB_INDEX,
                                            'WEIGHTED_BY_CITY_GROUP_CREDIT_SUB_INDEX': WEIGHTED_BY_CITY_GROUP_CREDIT_SUB_INDEX}
                                            ).T    

WEIGHTED_BY_CITY_GROUP_INDEX_DF.loc['WEIGHTED_BY_CITY_GROUP_HEADLINE_INDEX'] = WEIGHTED_BY_CITY_GROUP_INDEX_DF.mean() 
WEIGHTED_BY_CITY_GROUP_INDEX_DF 

City_group,Hong Kong,Guangzhou,Shenzhen,Foshan,Dongguan,Macao and other GBA cities
WEIGHTED_BY_CITY_GROUP_PERFORMANCE_SUB_INDEX,48.228901,47.064191,43.945178,46.491119,52.223423,50.739655
WEIGHTED_BY_CITY_GROUP_EXPECTATION_SUB_INDEX,50.411870,52.422887,55.117215,55.149086,50.833189,47.456493
WEIGHTED_BY_CITY_GROUP_CREDIT_SUB_INDEX,51.338447,53.303166,46.712572,46.141887,47.583718,47.044115
WEIGHTED_BY_CITY_GROUP_HEADLINE_INDEX,49.993073,50.930081,48.591655,49.260697,50.213443,48.413421


In [100]:
# export WEIGHTED_BY_CITY_GROUP_INDEX result
WEIGHTED_BY_CITY_GROUP_INDEX_DF.to_excel(f"{store_folder}/reference_Part4.4_WEIGHTED_BY_CITY_GROUP_INDEX.xlsx", na_rep='N.A.')

#Part 5: Compile weighted-index by City Group and then by Industry

In [101]:
# just have a look
City_group_rows

['Hong Kong',
 'Guangzhou',
 'Shenzhen',
 'Foshan',
 'Dongguan',
 'Macao and other GBA cities']

In [102]:
# just have a look
WEIGHTED_SUM_score

,Manufacturing & Trading,Retail & Wholesale,Financial Services,Professional Services,Innovation & Technology
Hong Kong,2157.68,512.484,232.334,371.657,22.8198
Guangzhou,2072.84,621.374,16.1453,38.3634,27.8681
Shenzhen,5824.69,777.172,35.25,24.3364,243.923
Foshan,1132.45,25.5482,0.506747,0.768006,NaN
Dongguan,1210.36,46.9973,0.532842,2.69479,NaN
Macao and other GBA cities,2212.18,102.525,2.78239,8.19989,NaN


##Part 5.1: WEIGHTED_SUM_score_1D

In [103]:
WEIGHTED_SUM_score_1D = WEIGHTED_SUM_score.values.ravel()

In [104]:
WEIGHTED_SUM_score_1D

array([2157.678354533654, 512.484074160933, 232.33361651808275,
       371.6570700948316, 22.819810600930808, 2072.837708713877,
       621.3741242687788, 16.145301522393282, 38.36337139591804,
       27.868077305472887, 5824.6945997813355, 777.1718722233414,
       35.24999554083112, 24.336415554902278, 243.92323094861382,
       1132.4531916088351, 25.54815406923933, 0.5067465573564726,
       0.7680062890796606, nan, 1210.3631956716333, 46.9973410141141,
       0.532842274242189, 2.6947920210604894, nan, 2212.1762250327833,
       102.52457662668829, 2.782394540656059, 8.199887034388448, nan],
      dtype=object)

In [105]:
WEIGHTED_SUM_score_1D.shape

(30,)

In [106]:
# export WEIGHTED_SUM_score_CITY_GROUP_1D
pd.DataFrame(WEIGHTED_SUM_score_1D).T.to_excel(f"{store_folder}/reference_Part5.1_WEIGHTED_SUM_score_1D.xlsx", na_rep='N.A.',index=False)
pd.DataFrame(WEIGHTED_SUM_score_1D).T

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,2157.68,512.484,232.334,371.657,22.8198,2072.84,621.374,16.1453,38.3634,27.8681,...,1210.36,46.9973,0.532842,2.69479,NaN,2212.18,102.525,2.78239,8.19989,NaN


##Part 5.2: responses_weighted_sum_byquestion_city_GROUP then further by industry

In [107]:
# have a look from part 2.2
DF_INDEX_WEIGHTED

,Q01,Q02,City,City_group,Industry,weighting_score,Q03A,Q04A,Q05A,Q06A,...,Q08B,Q09B,Q10B,Q11B,Q12B,Q13B,Q14B,Q15B,Q16B,Q17B
0,11,4,Zhaoqing,Macao and other GBA cities,Professional Services,0.073213,-0.073213,0.000000,0.000000,0.073213,...,0.073213,NaN,0.073213,NaN,-0.073213,-0.073213,0.073213,0.073213,-0.073213,0.000000
1,9,4,Jiangmen,Macao and other GBA cities,Professional Services,0.073213,0.000000,0.073213,-0.073213,0.073213,...,0.073213,0.000000,-0.073213,-0.073213,NaN,0.073213,0.000000,-0.073213,0.073213,0.073213
2,10,3,Zhuhai,Macao and other GBA cities,Financial Services,0.022259,-0.022259,0.022259,0.022259,0.022259,...,0.000000,-0.022259,0.022259,0.022259,0.022259,0.022259,0.022259,0.000000,0.000000,0.022259
3,4,5,Shenzhen,Shenzhen,Innovation & Technology,11.087420,11.087420,0.000000,0.000000,NaN,...,11.087420,11.087420,-11.087420,11.087420,11.087420,-11.087420,-11.087420,11.087420,11.087420,-11.087420
4,9,3,Jiangmen,Macao and other GBA cities,Financial Services,0.022259,0.022259,0.000000,0.000000,-0.022259,...,-0.022259,0.000000,0.022259,-0.022259,0.000000,-0.022259,-0.022259,-0.022259,0.022259,0.022259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,11,4,Zhaoqing,Macao and other GBA cities,Professional Services,0.073213,0.073213,0.073213,0.073213,0.073213,...,0.073213,0.073213,-0.073213,NaN,-0.073213,0.073213,-0.073213,-0.073213,0.073213,0.000000
996,1,3,Hong Kong,Hong Kong,Financial Services,10.101462,10.101462,-10.101462,0.000000,NaN,...,-10.101462,NaN,10.101462,10.101462,-10.101462,10.101462,0.000000,10.101462,-10.101462,-10.101462
997,11,1,Zhaoqing,Macao and other GBA cities,Manufacturing & Trading,18.747256,0.000000,-18.747256,-18.747256,18.747256,...,-18.747256,NaN,18.747256,18.747256,18.747256,0.000000,18.747256,-18.747256,0.000000,-18.747256
998,7,4,Huizhou,Macao and other GBA cities,Professional Services,0.073213,0.000000,0.073213,0.073213,0.000000,...,-0.073213,NaN,0.073213,0.000000,0.073213,-0.073213,0.000000,0.073213,0.073213,-0.073213


In [108]:
bycity_GROUP_then_industry = DF_INDEX_WEIGHTED.groupby(['City_group','Industry']).sum()[selected_columns_list]
bycity_GROUP_then_industry

Q03A        Q04A  \
City_group                 Industry                                            
Dongguan                   Financial Services       0.000000e+00   -0.122964   
                           Innovation & Technology  0.000000e+00    0.000000   
                           Manufacturing & Trading -7.564770e+01  226.943099   
                           Professional Services   -6.124527e-01   -0.489962   
                           Retail & Wholesale       2.043363e+00   18.390264   
Foshan                     Financial Services       6.938894e-18    0.022032   
                           Innovation & Technology  0.000000e+00    0.000000   
                           Manufacturing & Trading -2.123350e+02 -212.334973   
                           Professional Services   -2.400020e-01    0.192002   
                           Retail & Wholesale       3.930485e+00   -5.895728   
Guangzhou                  Financial Services      -2.690884e+00    2.690884   
                           Innovation & Technology -3.483510e+00   19.159303   
                           Manufacturing & Trading  4.145675e+02    0.000000   
                           Professional Services    1.475514e+00    4.426543   
                           Retail & Wholesale      -1.035624e+02   25.890589   
Hong Kong                  Financial Services      -2.020292e+01   30.304385   
                           Innovation & Technology  1.629986e+00   -4.889959   
                           Manufacturing & Trading  3.406861e+02    0.000000   
                           Professional Services    4.459885e+01   59.465131   
                           Retail & Wholesale       7.884370e+01 -197.109259   
Macao and other GBA cities Financial Services      -1.558141e-01    0.111296   
                           Innovation & Technology  0.000000e+00    0.000000   
                           Manufacturing & Trading  2.624616e+02  131.230793   
                           Professional Services    2.196398e-01    0.146427   
                           Retail & Wholesale       1.281557e+01   13.801385   
Shenzhen                   Financial Services       2.203125e+00    6.609374   
                           Innovation & Technology  6.652452e+01   77.611937   
                           Manufacturing & Trading -8.961069e+02    0.000000   
                           Professional Services   -4.563078e+00    1.521026   
                           Retail & Wholesale      -1.371480e+02   91.431985   

                                                          Q05A         Q06A  \
City_group                 Industry                                           
Dongguan                   Financial Services        -0.122964     0.000000   
                           Innovation & Technology    0.000000     0.000000   
                           Manufacturing & Trading   75.647700   226.943099   
                           Professional Services     -0.367472    -0.122491   
                           Retail & Wholesale       -18.390264     4.086725   
Foshan                     Financial Services         0.022032    -0.044065   
                           Innovation & Technology    0.000000     0.000000   
                           Manufacturing & Trading  283.113298   141.556649   
                           Professional Services      0.144001     0.048000   
                           Retail & Wholesale        -1.965243    -1.965243   
Guangzhou                  Financial Services         4.036325     0.000000   
                           Innovation & Technology   -6.967019     6.967019   
                           Manufacturing & Trading -207.283771  -414.567542   
                           Professional Services     -1.475514    -5.902057   
                           Retail & Wholesale      -258.905885   -25.890589   
Hong Kong                  Financial Services        40.405846    40.405846   
                           Innovation & Technology   -8.149932     3.259973  

In [109]:
# adjust multi-index position
bycity_GROUP_then_industry = bycity_GROUP_then_industry.reindex(City_group_rows, level='City_group')

In [110]:
# adjust multi-index position
bycity_GROUP_then_industry = bycity_GROUP_then_industry.reindex(industry_names, level='Industry')

In [111]:
bycity_GROUP_then_industry

Q03A        Q04A  \
City_group                 Industry                                            
Hong Kong                  Manufacturing & Trading  3.406861e+02    0.000000   
                           Retail & Wholesale       7.884370e+01 -197.109259   
                           Financial Services      -2.020292e+01   30.304385   
                           Professional Services    4.459885e+01   59.465131   
                           Innovation & Technology  1.629986e+00   -4.889959   
Guangzhou                  Manufacturing & Trading  4.145675e+02    0.000000   
                           Retail & Wholesale      -1.035624e+02   25.890589   
                           Financial Services      -2.690884e+00    2.690884   
                           Professional Services    1.475514e+00    4.426543   
                           Innovation & Technology -3.483510e+00   19.159303   
Shenzhen                   Manufacturing & Trading -8.961069e+02    0.000000   
                           Retail & Wholesale      -1.371480e+02   91.431985   
                           Financial Services       2.203125e+00    6.609374   
                           Professional Services   -4.563078e+00    1.521026   
                           Innovation & Technology  6.652452e+01   77.611937   
Foshan                     Manufacturing & Trading -2.123350e+02 -212.334973   
                           Retail & Wholesale       3.930485e+00   -5.895728   
                           Financial Services       6.938894e-18    0.022032   
                           Professional Services   -2.400020e-01    0.192002   
                           Innovation & Technology  0.000000e+00    0.000000   
Dongguan                   Manufacturing & Trading -7.564770e+01  226.943099   
                           Retail & Wholesale       2.043363e+00   18.390264   
                           Financial Services       0.000000e+00   -0.122964   
                           Professional Services   -6.124527e-01   -0.489962   
                           Innovation & Technology  0.000000e+00    0.000000   
Macao and other GBA cities Manufacturing & Trading  2.624616e+02  131.230793   
                           Retail & Wholesale       1.281557e+01   13.801385   
                           Financial Services      -1.558141e-01    0.111296   
                           Professional Services    2.196398e-01    0.146427   
                           Innovation & Technology  0.000000e+00    0.000000   

                                                          Q05A         Q06A  \
City_group                 Industry                                           
Hong Kong                  Manufacturing & Trading  227.124037  -113.562019   
                           Retail & Wholesale       197.109259  -118.265556   
                           Financial Services        40.405846    40.405846   
                           Professional Services     44.598848    44.598848   
                           Innovation & Technology   -8.149932     3.259973   
Guangzhou                  Manufacturing & Trading -207.283771  -414.567542   
                           Retail & Wholesale      -258.905885   -25.890589   
                           Financial Services         4.036325     0.000000   
                           Professional Services     -1.475514    -5.902057   
                           Innovation & Technology   -6.967019     6.967019   
Shenzhen                   Manufacturing & Trading -448.053431 -1792.213723   
                           Retail & Wholesale       182.863970  -137.147977   
                           Financial Services         2.203125     2.203125   
                           Professional Services      3.042052     4.563078   
                           Innovation & Technology  -77.611937    44.349678   
Foshan                     Manufacturing & Trading  283.113298   141.556649   
                           Retail & Wholesale        -1.965243    -1.965243  

In [112]:
# transpose the result
responses_weighted_sum_byquestion_city_GROUP_then_industry = bycity_GROUP_then_industry.T

In [113]:
responses_weighted_sum_byquestion_city_GROUP_then_industry

City_group               Hong Kong                                        \
Industry   Manufacturing & Trading Retail & Wholesale Financial Services   
Q03A                    340.686056       7.884370e+01         -20.202923   
Q04A                      0.000000      -1.971093e+02          30.304385   
Q05A                    227.124037       1.971093e+02          40.405846   
Q06A                   -113.562019      -1.182656e+02          40.405846   
Q07A                   -113.562019      -1.421085e-14           0.000000   
Q08A                   -794.934131       7.884370e+01         -10.101462   
Q09A                      0.000000       0.000000e+00          10.101462   
Q10A                   -340.686056      -2.759530e+02         -20.202923   
Q03B                    454.248075      -1.182656e+02           0.000000   
Q04B                      0.000000      -7.884370e+01          20.202923   
Q05B                    113.562019       0.000000e+00          30.304385   
Q06B                    340.686056      -1.421085e-14         -10.101462   
Q07B                   -908.496149       1.576874e+02          20.202923   
Q08B                    454.248075       1.182656e+02          50.507308   
Q09B                    113.562019      -7.884370e+01           0.000000   
Q10B                   -340.686056       3.942185e+01         -10.101462   
Q13A                   -340.686056      -7.884370e+01         -10.101462   
Q14A                    340.686056       7.884370e+01          10.101462   
Q15A                   -113.562019      -3.942185e+01          10.101462   
Q16A                    227.124037      -3.942185e+01         -30.304385   
Q17A                    340.686056       1.182656e+02         -50.507308   

City_group                                                \
Industry   Professional Services Innovation & Technology   
Q03A                   44.598848                1.629986   
Q04A                   59.465131               -4.889959   
Q05A                   44.598848               -8.149932   
Q06A                   44.598848                3.259973   
Q07A                  -59.465131                1.629986   
Q08A                  -74.331414                1.629986   
Q09A                    0.000000                1.629986   
Q10A                  -29.732566                0.000000   
Q03B                  -29.732566                4.889959   
Q04B                   59.465131                1.629986   
Q05B                   14.866283               -4.889959   
Q06B                  -14.866283               -1.629986   
Q07B                  -59.465131                0.000000   
Q08B                  -44.598848               -4.889959   
Q09B                  -29.732566               -1.629986   
Q10B                  -44.598848                4.889959   
Q13A                  104.063980               -1.629986   
Q14A                   14.866283                0.000000   
Q15A                  -14.866283                1.629986   
Q16A                  178.395394              -14.669878   
Q17A                  -14.866283                0.000000   

City_group               Guangzhou                                        \
Industry   Manufacturing & Trading Retail & Wholesale Financial Services   
Q03A                    414.567542      -1.035624e+02          -2.690884   
Q04A                      0.000000       2.589059e+01           2.690884   
Q05A                   -207.283771      -2.589059e+02           4.036325   
Q06A                   -414.567542      -2.589059e+01           0.000000   
Q07A                   -621.851313       7.767177e+01          -5.381767   
Q08A                      0.000000       1.294529e+02           0.000000   
Q09A                   -414.567542       5.178118e+01           2.690884   
Q10A                      0.000000      -7.105427e-15          -2.690884   
Q03B                    621.851313      -1.812341e+02           6.727209   
Q04B                   -207.283771

In [114]:
# export responses_weighted_sum_byquestion_city_group_then_industry
responses_weighted_sum_byquestion_city_GROUP_then_industry.to_excel(f"{store_folder}/reference_Part5.2_responses_weighted_sum_byquestion_city_GROUP_then_industry.xlsx", na_rep='N.A.')

In [115]:
# divide each column by according city weighted sum value(WEIGHTED_SUM_score_CITY_GROUP_1D)
responses_weighted_sum_byquestion_city_GROUP_then_industry / WEIGHTED_SUM_score_1D

City_group               Hong Kong                                        \
Industry   Manufacturing & Trading Retail & Wholesale Financial Services   
Q03A                      0.157895       1.538462e-01          -0.086957   
Q04A                      0.000000      -3.846154e-01           0.130435   
Q05A                      0.105263       3.846154e-01           0.173913   
Q06A                     -0.052632      -2.307692e-01           0.173913   
Q07A                     -0.052632      -2.772936e-17           0.000000   
Q08A                     -0.368421       1.538462e-01          -0.043478   
Q09A                      0.000000       0.000000e+00           0.043478   
Q10A                     -0.157895      -5.384615e-01          -0.086957   
Q03B                      0.210526      -2.307692e-01           0.000000   
Q04B                      0.000000      -1.538462e-01           0.086957   
Q05B                      0.052632       0.000000e+00           0.130435   
Q06B                      0.157895      -2.772936e-17          -0.043478   
Q07B                     -0.421053       3.076923e-01           0.086957   
Q08B                      0.210526       2.307692e-01           0.217391   
Q09B                      0.052632      -1.538462e-01           0.000000   
Q10B                     -0.157895       7.692308e-02          -0.043478   
Q13A                     -0.157895      -1.538462e-01          -0.043478   
Q14A                      0.157895       1.538462e-01           0.043478   
Q15A                     -0.052632      -7.692308e-02           0.043478   
Q16A                      0.105263      -7.692308e-02          -0.130435   
Q17A                      0.157895       2.307692e-01          -0.217391   

City_group                                                \
Industry   Professional Services Innovation & Technology   
Q03A                        0.12                0.071429   
Q04A                        0.16               -0.214286   
Q05A                        0.12               -0.357143   
Q06A                        0.12                0.142857   
Q07A                       -0.16                0.071429   
Q08A                       -0.20                0.071429   
Q09A                        0.00                0.071429   
Q10A                       -0.08                0.000000   
Q03B                       -0.08                0.214286   
Q04B                        0.16                0.071429   
Q05B                        0.04               -0.214286   
Q06B                       -0.04               -0.071429   
Q07B                       -0.16                0.000000   
Q08B                       -0.12               -0.214286   
Q09B                       -0.08               -0.071429   
Q10B                       -0.12                0.214286   
Q13A                        0.28               -0.071429   
Q14A                        0.04                0.000000   
Q15A                       -0.04                0.071429   
Q16A                        0.48               -0.642857   
Q17A                       -0.04                0.000000   

City_group               Guangzhou                                        \
Industry   Manufacturing & Trading Retail & Wholesale Financial Services   
Q03A                           0.2      -1.666667e-01          -0.166667   
Q04A                           0.0       4.166667e-02           0.166667   
Q05A                          -0.1      -4.166667e-01           0.250000   
Q06A                          -0.2      -4.166667e-02           0.000000   
Q07A                          -0.3       1.250000e-01          -0.333333   
Q08A                           0.0       2.083333e-01           0.000000   
Q09A                          -0.2       8.333333e-02           0.166667   
Q10A                           0.0      -1.143502e-17          -0.166667   
Q03B                           0.3      -2.916667e-01           0.416667   
Q04B                          -0.1

##Part 5.3: WEIGHTED_INDEX_BYQUESTION_CITY_GROUP_furtherby_INDUSTRY

In [116]:
WEIGHTED_INDEX_BYQUESTION_CITY_GROUP_furtherby_INDUSTRY = (responses_weighted_sum_byquestion_city_GROUP_then_industry / WEIGHTED_SUM_score_1D * 100 + 100)/2
WEIGHTED_INDEX_BYQUESTION_CITY_GROUP_furtherby_INDUSTRY

City_group               Hong Kong                                        \
Industry   Manufacturing & Trading Retail & Wholesale Financial Services   
Q03A                     57.894737          57.692308          45.652174   
Q04A                     50.000000          30.769231          56.521739   
Q05A                     55.263158          69.230769          58.695652   
Q06A                     47.368421          38.461538          58.695652   
Q07A                     47.368421          50.000000          50.000000   
Q08A                     31.578947          57.692308          47.826087   
Q09A                     50.000000          50.000000          52.173913   
Q10A                     42.105263          23.076923          45.652174   
Q03B                     60.526316          38.461538          50.000000   
Q04B                     50.000000          42.307692          54.347826   
Q05B                     52.631579          50.000000          56.521739   
Q06B                     57.894737          50.000000          47.826087   
Q07B                     28.947368          65.384615          54.347826   
Q08B                     60.526316          61.538462          60.869565   
Q09B                     52.631579          42.307692          50.000000   
Q10B                     42.105263          53.846154          47.826087   
Q13A                     42.105263          42.307692          47.826087   
Q14A                     57.894737          57.692308          52.173913   
Q15A                     47.368421          46.153846          52.173913   
Q16A                     55.263158          46.153846          43.478261   
Q17A                     57.894737          61.538462          39.130435   

City_group                                                \
Industry   Professional Services Innovation & Technology   
Q03A                        56.0               53.571429   
Q04A                        58.0               39.285714   
Q05A                        56.0               32.142857   
Q06A                        56.0               57.142857   
Q07A                        42.0               53.571429   
Q08A                        40.0               53.571429   
Q09A                        50.0               53.571429   
Q10A                        46.0               50.000000   
Q03B                        46.0               60.714286   
Q04B                        58.0               53.571429   
Q05B                        52.0               39.285714   
Q06B                        48.0               46.428571   
Q07B                        42.0               50.000000   
Q08B                        44.0               39.285714   
Q09B                        46.0               46.428571   
Q10B                        44.0               60.714286   
Q13A                        64.0               46.428571   
Q14A                        52.0               50.000000   
Q15A                        48.0               53.571429   
Q16A                        74.0               17.857143   
Q17A                        48.0               50.000000   

City_group               Guangzhou                                        \
Industry   Manufacturing & Trading Retail & Wholesale Financial Services   
Q03A                          60.0          41.666667          41.666667   
Q04A                          50.0          52.083333          58.333333   
Q05A                          45.0          29.166667          62.500000   
Q06A                          40.0          47.916667          50.000000   
Q07A                          35.0          56.250000          33.333333   
Q08A                          50.0          60.416667          50.000000   
Q09A                          40.0          54.166667          58.333333   
Q10A                          50.0          50.000000          41.666667   
Q03B                          65.0          35.416667          70.833333   
Q04B                          45.0

In [117]:
# Q13, Q14 are related to financing cost, need to adjusted to 100-X
WEIGHTED_INDEX_BYQUESTION_CITY_GROUP_furtherby_INDUSTRY.loc['Q13A'] = 100-WEIGHTED_INDEX_BYQUESTION_CITY_GROUP_furtherby_INDUSTRY.loc['Q13A']
WEIGHTED_INDEX_BYQUESTION_CITY_GROUP_furtherby_INDUSTRY.loc['Q14A'] = 100-WEIGHTED_INDEX_BYQUESTION_CITY_GROUP_furtherby_INDUSTRY.loc['Q14A']
WEIGHTED_INDEX_BYQUESTION_CITY_GROUP_furtherby_INDUSTRY

City_group               Hong Kong                                        \
Industry   Manufacturing & Trading Retail & Wholesale Financial Services   
Q03A                     57.894737          57.692308          45.652174   
Q04A                     50.000000          30.769231          56.521739   
Q05A                     55.263158          69.230769          58.695652   
Q06A                     47.368421          38.461538          58.695652   
Q07A                     47.368421          50.000000          50.000000   
Q08A                     31.578947          57.692308          47.826087   
Q09A                     50.000000          50.000000          52.173913   
Q10A                     42.105263          23.076923          45.652174   
Q03B                     60.526316          38.461538          50.000000   
Q04B                     50.000000          42.307692          54.347826   
Q05B                     52.631579          50.000000          56.521739   
Q06B                     57.894737          50.000000          47.826087   
Q07B                     28.947368          65.384615          54.347826   
Q08B                     60.526316          61.538462          60.869565   
Q09B                     52.631579          42.307692          50.000000   
Q10B                     42.105263          53.846154          47.826087   
Q13A                     57.894737          57.692308          52.173913   
Q14A                     42.105263          42.307692          47.826087   
Q15A                     47.368421          46.153846          52.173913   
Q16A                     55.263158          46.153846          43.478261   
Q17A                     57.894737          61.538462          39.130435   

City_group                                                \
Industry   Professional Services Innovation & Technology   
Q03A                        56.0               53.571429   
Q04A                        58.0               39.285714   
Q05A                        56.0               32.142857   
Q06A                        56.0               57.142857   
Q07A                        42.0               53.571429   
Q08A                        40.0               53.571429   
Q09A                        50.0               53.571429   
Q10A                        46.0               50.000000   
Q03B                        46.0               60.714286   
Q04B                        58.0               53.571429   
Q05B                        52.0               39.285714   
Q06B                        48.0               46.428571   
Q07B                        42.0               50.000000   
Q08B                        44.0               39.285714   
Q09B                        46.0               46.428571   
Q10B                        44.0               60.714286   
Q13A                        36.0               53.571429   
Q14A                        48.0               50.000000   
Q15A                        48.0               53.571429   
Q16A                        74.0               17.857143   
Q17A                        48.0               50.000000   

City_group               Guangzhou                                        \
Industry   Manufacturing & Trading Retail & Wholesale Financial Services   
Q03A                          60.0          41.666667          41.666667   
Q04A                          50.0          52.083333          58.333333   
Q05A                          45.0          29.166667          62.500000   
Q06A                          40.0          47.916667          50.000000   
Q07A                          35.0          56.250000          33.333333   
Q08A                          50.0          60.416667          50.000000   
Q09A                          40.0          54.166667          58.333333   
Q10A                          50.0          50.000000          41.666667   
Q03B                          65.0          35.416667          70.833333   
Q04B                          45.0

In [118]:
# export to excel for reference
WEIGHTED_INDEX_BYQUESTION_CITY_GROUP_furtherby_INDUSTRY.to_excel(f"{store_folder}/reference_Part5.3_WEIGHTED_INDEX_BYQUESTION_CITY_GROUP_furtherby_INDUSTRY.xlsx", na_rep='N.A.')

##Part 5.4: WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_INDEX

In [119]:
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_PERFORMANCE_SUB_INDEX = WEIGHTED_INDEX_BYQUESTION_CITY_GROUP_furtherby_INDUSTRY.loc[performance_columns].mean()

WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_EXPECTATION_SUB_INDEX = WEIGHTED_INDEX_BYQUESTION_CITY_GROUP_furtherby_INDUSTRY.loc[expectation_columns].mean()

WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_CREDIT_SUB_INDEX = WEIGHTED_INDEX_BYQUESTION_CITY_GROUP_furtherby_INDUSTRY.loc[credit_columns].mean()

WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_INDEX_DF = pd.DataFrame({
                                            'WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_PERFORMANCE_SUB_INDEX': WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_PERFORMANCE_SUB_INDEX,
                                            'WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_EXPECTATION_SUB_INDEX': WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_EXPECTATION_SUB_INDEX,
                                            'WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_CREDIT_SUB_INDEX': WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_CREDIT_SUB_INDEX}
                                            ).T    

WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_INDEX_DF.loc['WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_HEADLINE_INDEX'] = WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_INDEX_DF.mean() 
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_INDEX_DF 

City_group                                                       Hong Kong  \
Industry                                           Manufacturing & Trading   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_PERFO...               47.697368   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_EXPEC...               50.657895   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_CREDI...               52.105263   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_HEADL...               50.153509   

City_group                                                             \
Industry                                           Retail & Wholesale   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_PERFO...          47.115385   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_EXPEC...          50.480769   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_CREDI...          50.769231   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_HEADL...          49.455128   

City_group                                                             \
Industry                                           Financial Services   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_PERFO...          51.902174   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_EXPEC...          52.717391   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_CREDI...          46.956522   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_HEADL...          50.525362   

City_group                                                                \
Industry                                           Professional Services   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_PERFO...                  50.5   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_EXPEC...                  47.5   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_CREDI...                  50.8   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_HEADL...                  49.6   

City_group                                                                  \
Industry                                           Innovation & Technology   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_PERFO...               49.107143   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_EXPEC...               49.553571   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_CREDI...               45.000000   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_HEADL...               47.886905   

City_group                                                       Guangzhou  \
Industry                                           Manufacturing & Trading   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_PERFO...               46.250000   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_EXPEC...               53.125000   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_CREDI...               53.000000   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_HEADL...               50.791667   

City_group                                                             \
Industry                                           Retail & Wholesale   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_PERFO...          48.958333   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_EXPEC...          50.260417   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_CREDI...          55.000000   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_HEADL...          51.406250   

City_group                                                             \
Industry                                           Financial Services   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_PERFO...          49.479167   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_EXPEC...          61.458333   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_CREDI...          56.666667   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_HEADL...          55.868056   

City_group                                                                \
Industry                                           Professional Services   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_PERFO...             52.403846   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_EXPEC...             51.442308   
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_CREDI...             45.000000   


In [120]:
# export WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_INDEX result
WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_INDEX_DF .to_excel(f"{store_folder}/reference_Part5.4_WEIGHTED_BY_CITY_GROUP_furtherby_INDUSTRY_INDEX.xlsx", na_rep='N.A.')